In [ ]:
import random
import time
import numpy as np

In [ ]:
def VAR_init(var, X): #_init
    
    if var in [0]: X[var] = random.randint(1, 512) # of hidden neurons
    elif var in [1]: X[var] = random.randint(1, 99) # dropout ratio
   
    elif var in [2]:  X[var] = random.randint(1, 512) # of hidden neurons
    elif var in [3]:  X[var] = random.randint(1, 99) # dropout ratio
          
    elif var in [4]:  X[var] = random.randint(1, 512) # of hidden neurons
    elif var in [5]:  X[var] = random.randint(1, 99) # dropout ratio 
        
    elif var in [6]:  X[var] = random.randint(1, 99)

In [ ]:
Nrun, Ngen, Nsol, Nvar = 30, 10, 5, 7
pX = np.zeros([Nsol, Nvar], dtype = int)

for run in range(0, Nrun):
    X  = np.zeros([Nsol, Nvar], dtype = int)

for sol in range(0, Nsol):
    print("----- sol_{} -----".format(sol))
    for var in range(Nvar):
        VAR_init(var, X[sol]) #_init           
        pX[sol][var] = X[sol][var] # 每一個解的pbest
    
    print(X[sol])

In [ ]:
def ORIGIN(X): #_init
    global run, df
    
#     print("*****ORIGIN*****\n")
    print("=============== ORIGIN ===============")
    
    
    X[0], X[1] = 10, 20
    X[2], X[3] = 40, 20
    X[4], X[5] = 100, 20
    X[6] = 50        
    
    
    t0   = time.process_time()
    F[0] = Model(X)
    t1   = time.process_time()
    pT[0] = (t1 - t0)/60
        
    new_row = {'F': F[0], 'time': pT[0], 
               'pbest': F[0], 'pbest(var0)': X[0], 'pbest(var1)': X[1], 'pbest(var2)': X[2], 
               'pbest(var3)': X[3],'pbest(var4)': X[4], 'pbest(var5)': X[5], 'pbest(var6)': X[6],
                'gbest(var0)': X[0], 'gbest(var1)': X[1], 'gbest(var2)': X[2], 
                'gbest(var3)': X[3],'gbest(var4)': X[4], 'gbest(var5)': X[5], 'gbest(var6)': X[6]}
    
    df = df.append(new_row, ignore_index=True)
    
    print("Origin_sol_set: ({}, {}, {}, {}, {}, {}, {})".format(X[0], X[1], X[2], X[3], X[4], X[5], X[6]))
    print("sol_fitness: {:^.4f}".format(F[0]))
    print("time: {:^.2f} mins\n".format(pT[0]))



In [ ]:
def SSO_init(X, F, pX, pF, Nsol, Nvar):
    
    global gBest, df
    print("*****SSO_INIT*****\n")
    print("=============== Generation_0 ===============")
    
    for sol in range(0, Nsol):
        print("----- sol_{} -----".format(sol))
        for var in range(Nvar):
            VAR_init(var, X[sol]) #_init           
            pX[sol][var] = X[sol][var] # 每一個解的pbest
        #print(X[sol])
        
        t0 = time.process_time()
        
        
        F[sol] = Model(X[sol])
        pF[sol] = F[sol]
                
        t1 = time.process_time()
        pT[sol] = (t1 - t0)/60
        
        print("Initialize sol_set_{}:({}, {}, {}, {}, {}, {}, {})".format(sol, X[sol][0], X[sol][1], X[sol][2], X[sol][3], X[sol][4], X[sol][5], X[sol][6]))
        print("Initialize pbest_fitness_{}: {:^.4f}".format(sol, pF[sol]))
        print("time: {:^.2f} mins\n".format(pT[sol]))
        
        if F[sol] > F[gBest]:
            gBest = sol   
        
        new_row = {'gen': gen, 'sol': sol, 'F': F[sol], 'time': pT[sol], 
                   'pbest': pF[sol], 'pbest(var0)': pX[sol][0], 'pbest(var1)': pX[sol][1], 'pbest(var2)': pX[sol][2], 
                   'pbest(var3)': pX[sol][3],'pbest(var4)': pX[sol][4], 'pbest(var5)': pX[sol][5], 'pbest(var6)': pX[sol][6],
                   'gbest(gen)': genBest, 'gbest(sol)': gBest, 'gbest(F)': pF[gBest], 
                    'gbest(var0)': pX[gBest][0], 'gbest(var1)': pX[gBest][1], 'gbest(var2)': pX[gBest][2], 
                   'gbest(var3)': pX[gBest][3],'gbest(var4)': pX[gBest][4], 'gbest(var5)': pX[gBest][5]
                      ,'gbest(var6)': pX[gBest][6]}
        df = df.append(new_row, ignore_index=True)
        
    print("Initialize gbest_fitness: {:^.4f}\n".format(F[gBest])) 

In [ ]:
def SSO_UPDATE(sol):
    
    global gen, gBest, genBest, Cg, Cp, Cw, df
    
#     print("*****SSO_UPDATE*****\n")
    
    for var in range(Nvar):        
        rnd = np.random.rand(1)
        
#         print("rnd: {}\n".format(rnd))
        
        if   rnd < Cg:
            X[sol][var] = pX[gBest][var]
        elif rnd < Cp:
            X[sol][var] = pX[sol][var]   
        elif rnd > Cw:
            VAR_init(var, X[sol]) 
        #print(var, X[sol][var])
    
    
    
    t0 = time.process_time()
    F[sol] = Model(X[sol])
    t1 = time.process_time()
    Xtime = (t1 - t0)/60

    if F[sol] > pF[sol]: # check personal best
        pF[sol] = F[sol]
        pT[sol] = Xtime
        
        for var in range(0, Nvar): 
            pX[sol][var] = X[sol][var]
            
        if F[sol] > pF[gBest]: # check global best
            gBest = sol
            genBest = gen
    
    print("sol_set:({}, {}, {}, {}, {}, {}, {})".format(X[sol][0], X[sol][1], X[sol][2], X[sol][3], X[sol][4], X[sol][5], X[sol][6]))
    print("sol_fitness: {:^.4f}".format(F[sol]))
    print("gbest_fitness: {:^.4f}\n".format(pF[gBest]))
    print("time: {:^.2f} mins\n".format(Xtime))
    
    
    new_row = {'gen': gen, 'sol': sol, 'F': F[sol], 'time': Xtime, 
               'pbest': pF[sol], 'pbest(var0)': pX[sol][0], 'pbest(var1)': pX[sol][1], 'pbest(var2)': pX[sol][2], 
               'pbest(var3)': pX[sol][3],'pbest(var4)': pX[sol][4], 'pbest(var5)': pX[sol][5], 'pbest(var6)': pX[sol][6],
               'gbest(gen)': genBest, 'gbest(sol)': gBest, 'gbest(F)': pF[gBest], 
                'gbest(var0)': pX[gBest][0], 'gbest(var1)': pX[gBest][1], 'gbest(var2)': pX[gBest][2], 
               'gbest(var3)': pX[gBest][3],'gbest(var4)': pX[gBest][4], 'gbest(var5)': pX[gBest][5]
                  ,'gbest(var6)': pX[gBest][6]}
    df = df.append(new_row, ignore_index=True)



In [ ]:
Nrun, Ngen, Nsol, Nvar = 30, 10, 5, 7
    
Cg, Cp, Cw = 0.2, 0.6, 0.7

F  = np.zeros([Nsol], dtype = float)
pF = np.zeros([Nsol], dtype = float)

In [8]:
import random

# 调用函数生成随机数
def generate_random_numbers(number_range):
    random_numbers = []
    for min_value, max_value in number_range:
        random_value = random.uniform(min_value, max_value)
        if random_value>1:
          random_value=int(random_value)
        random_numbers.append(random_value)

    return random_numbers
# 定义范围
random_number_range=[(0.001, 0.1), (1e-4, 1e-3),(0.2,0.8),(8,129),(1,5)]

rX = generate_random_numbers(random_number_range)
def random_select(n):
    return random.uniform(random_number_range[n][0],random_number_range[n][1])

In [9]:
def F(rX):
    fit = 100*(rX[0])+100*(+rX[1])+10*(rX[2])+rX[3]+(2*rX[4])

    return fit

In [22]:
import numpy as np
import time
import copy
import random
import os

Cg=0.1#GBEST區間
Cp=0.3#PBEST區間
Cw=0.6#前解區間

random.seed()
random_number_range=[(0.0005, 0.1), (1e-4, 1e-3),(0.2,0.7),(8,256),(1,4)]


#代數與每代幾組解
Njob=3 # 表連續型參數個數
Njob2=2 # 表離散型參數個數
start_time = time.time()
X, FX, pX, pF, gBest, genBest = [], [], [], [], 0, 0,
value_to_x_dict = {}
umax=np.array([0]*Njob, dtype=np.float64)
umin=np.array([1000]*Njob, dtype=np.float64)

#pX存gbest解pF存值
gen=0

#初始解
for sol in range(Nsol):
    rX = generate_random_numbers(random_number_range)
    X.append(rX) # add to the X2.
    pX.append(rX) # add to the X2.
    ##value表準確度?，F表函數，RX是你想更新的超參數
    value=F(rX)
    for job in range(Njob):
        if X[sol][job]>umax[job]:
            umax[job]=X[sol][job]
        if X[sol][job]<umin[job]:
            umin[job]=X[sol][job]
    FX.append(value)
    pF.append(value)
    if FX[sol]>FX[gBest]:
        gBest=sol
        pX[gBest]=rX
    print("gen&sol = {} {}, current sol = {}, fit = {}".format(gen,sol ,rX,value))
    value_to_x_dict[value] = (tuple(X[sol]),(gen,sol))

for gen in range(1,Ngen+1):
  for sol in range(Nsol):
    job=-1

    #sso
    while job<Njob+Njob2-1:
        job+=1
        rnd2=np.random.rand()
        if rnd2<Cg:
            X[sol][job]=pX[gBest][job]
            if X[sol][job]<random_number_range[job][0] or X[sol][job]>random_number_range[job][1]:
                X[sol][job]=int(random_select(job))
        elif rnd2<Cp:
            X[sol][job]=pX[sol][job]
            if X[sol][job]<random_number_range[job][0] or X[sol][job]>random_number_range[job][1]:
                X[sol][job]=int(random_select(job))
        elif rnd2<Cw:
            X[sol][job]=X[sol-1][job]
            if X[sol][job]<random_number_range[job][0] or X[sol][job]>random_number_range[job][1]:
                X[sol][job]=int(random_select(job))
        else:
            X[sol][job]=int(random_select(job))
            if X[sol][job]<random_number_range[job][0] or X[sol][job]>random_number_range[job][1]:
                X[sol][job]=int(random_select(job))
            
        #計算value
        ##value表目標值，F表函數，RX是你想更新的變數
        value=F(rX)
    #計算value結束
    value_to_x_dict[value] = (tuple(X[sol]),(gen,sol))
    #判斷大小
    #print("目前gbest",pF[gBest])
    if value < pF[sol]:
        pF[sol] = value
        if value < pF[gBest]:
            gBest=sol
            genBest=gen
            print("better than gbest, value = {}".format(value))
    if gen==1 and sol==0:
        print("initial sol, value = {}".format(value))

end_time = time.time()
sso_time1=end_time-start_time
print("\n==============================================================================================")
print("optimal sequence",value_to_x_dict[pF[gBest]][0])
print("optimal value:%f"%pF[gBest])
print("optimal generation number:", value_to_x_dict[pF[gBest]][1][0])
print("optimal sol number:", value_to_x_dict[pF[gBest]][1][1])
print("sso_time:", sso_time1)

gen&sol = 0 0, current sol = [0.06745241541174309, 0.0006423914413312234, 0.6118057706097544, 29, 1], fit = 43.92753839140497
gen&sol = 0 1, current sol = [0.042506599309622854, 0.0008643919083533431, 0.6654429351331603, 136, 2], fit = 150.99152847312922
gen&sol = 0 2, current sol = [0.0979205292371168, 0.0004502571541363759, 0.6904663196231908, 15, 3], fit = 37.741741835357224
gen&sol = 0 3, current sol = [0.009127632457547557, 0.00011159441676237284, 0.6282483535482797, 236, 1], fit = 245.20640622291378
gen&sol = 0 4, current sol = [0.009922904546396523, 0.0008977761407166636, 0.2889822844694472, 59, 2], fit = 66.97189091340579
initial sol, value = 66.97189091340579
better than gbest, value = 66.97189091340579
better than gbest, value = 63.91276324575475

optimal sequence (0, 0, 0, 130, 1)
optimal value:25.912763
optimal generation number: 4
optimal sol number: 3
sso_time: 0.0017058849334716797


In [34]:
value_to_x_dict

{175.94644850597055: ((0.08693225806940738,
   0.00048186131627968396,
   0.4205036567401832,
   159,
   2),
  (0, 0)),
 111.66720191588388: ((0.028442411588330774,
   0.00017282325625316014,
   0.5805678431425492,
   97,
   3),
  (0, 1)),
 161.73465287725753: ((0.07099586721670238,
   0.00025525617633195407,
   0.6609540537954102,
   146,
   1),
  (0, 2)),
 166.56480220953165: ((0.02039430045648605,
   0.0005692199895956256,
   0.3468450164923492,
   155,
   3),
  (0, 3)),
 51.95347477348011: ((0.0551629912019133,
   0.0007073300163984308,
   0.3366442651648932,
   37,
   3),
  (1, 4)),
 51.88274177184026: ((0.0551629912019133, 0, 0.3366442651648932, 37, 3),
  (1, 4)),
 52.72133568759316: ((0.0551629912019133, 0, 0.4205036567401832, 37, 3),
  (1, 4)),
 59.72133568759316: ((0.0551629912019133, 0, 0.4205036567401832, 44, 3),
  (1, 4)),
 55.72133568759316: ((0.0551629912019133, 0, 0.4205036567401832, 44, 1),
  (2, 4)),
 55.74686130522636: ((0.0551629912019133,
   0.00025525617633195407,
